## FileStreamWordCount

In [1]:
import findspark 
findspark.init(findspark.init("/Users/resitkadir/spark/spark-2.4.6/"))
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


from pyspark.streaming import *
from pyspark import SparkConf, SparkContext
#Scraming context aalim
sc = SparkContext("local[4]", "StreamingQuickExample")
#Steeaming context ,trigger lari 10 saniyede bir 
ssc = StreamingContext(sc, batchDuration=10)
#text deki leri line diye getirsin
lines = ssc.textFileStream("spark-streaming-test")
#uygulamam biz bitirene kadar. devam eder,baslatalim
words = lines.flatMap(lambda x: x.split(" "))
#kelimenin yanina rakam yazalim
mapped_words = words.map(lambda x: (x,1))
#kelime bazinda reduce edelim
wordcounts = mapped_words.reduceByKey(lambda x,y: x + y)

#siralama yapalim rakam ve kelimei degistirerek
#klasik rdd den farki sadece burasi .transfrom dur
sorted_wordcounts = wordcounts.map(lambda x: (x[1], x[0])) \
                                .transform(lambda x: x.sortByKey(False))

sorted_wordcounts.pprint(20)

ssc.start()

In [ ]:
#"spark-streaming-test" file icine kodu calistirdikdan sonra atilmali

In [2]:
ssc.awaitTermination()

-------------------------------------------
Time: 2020-07-12 02:57:40
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:57:50
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:58:00
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:58:10
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:58:20
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:58:30
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:58:40
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:58:50
-------------------------------------------

-------------------------------------------
Time: 2020-07-12 02:59:00
----------

KeyboardInterrupt: 

In [ ]:
ssc.stop()

## WindowOps

### SparkContext, StreamingContext ve FileStream Okuma

        .set("spark.executor.heartbeatInterval","3600s") \
    .set("spark.network.timeout", "4000s") \

In [1]:
import findspark 
findspark.init(findspark.init("/Users/resitkadir/spark/spark-2.4.6/"))
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from pyspark.streaming import *
from pyspark import SparkConf, SparkContext

conf = (SparkConf() \
        .set("spark.dirverimemory","2g") \
        .set("spark.executor.memory","4g") )

sc = SparkContext(appName="WindowOps", master="local[4]", conf=conf)

ssc = StreamingContext(sc, batchDuration=5)#5seconds

lines = ssc.textFileStream("spark-streaming-test")

#####################CHECKPOINT

#Count by windows icin onemli
ssc.checkpoint("checkpoint")

#okunacak dosya dizinini verelim

lines = ssc.textFileStream("spark-streaming-test")

################# Operasyon Başlangıç
#uygulamam biz bitirene kadar. devam eder,baslatalim
words = lines.flatMap(lambda x: x.split(" "))

mapped_words = words.map(lambda x: (x, 1))
#kelimenin yanina rakam yazalim



**WINDOWS OPS**

#windows duration 5ve slide duration birbirnin katlari olmasi lazim

#window =mapped_words.window(windowDuration=30,slideDuration=10)


30 saniye sureyle 10 tanesini goster

#YAZDIRALIM
#window.pprint()

**COUNT BY WINDOWS**

#ustteki window dan buraya 
#COUNT BY WINDOWS
#ustte verilenlerin cevrece icinde sayicakdir

#countByWindow = mapped_words.countByWindow(windowDuration=30,slideDuration=10)

#YAZDIR
#countByWindow.pprint()

#START
#ssc.start()
#bir kac defa kopyala kelime count artisini gor

### reduceByKeyAndWindow
    
    4 tane parametre alir
    *fonksiyon lambda  ve tersi fondksiyon reduce un tersi
    *


In [ ]:
#Dosyayi dizinine programi calistirdikdan sonra degisiklik yapilarak atmaliyiz

In [2]:
reduceByKeyAndWindow = mapped_words.reduceByKeyAndWindow(lambda x, y: int(x) + int(y), lambda x, y: int(x) - int(y), \
                                                         windowDuration=30, slideDuration=10) \
                    .map(lambda x: (x[1], x[0])) \
                    .transform(lambda x: x.sortByKey(False))
#sortu azalan yap
reduceByKeyAndWindow.pprint()
ssc.start()

In [3]:
ssc.stop()

-------------------------------------------
Time: 2020-07-12 04:33:50
-------------------------------------------



# READ FROM FILE

In [4]:
import findspark 
findspark.init("/Users/resitkadir/spark/spark-2.4.6/")
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder \
.master("local[4]") \
.appName("ReadFromFile") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory", "4g") \
.getOrCreate()

#dosyadan okduugmuz file ilebakalim
#eger kafkadan okusaydik readKafka olucakdu
#dizini aldik
lines = spark.readStream \
        .format("text") \
        .load("spark-streaming-test")

In [5]:
#digerlerinin aksine burada file sonradan eklemeye gerek yokdur

In [6]:
#kelimelere ayiralim
#belirlenen satira gore parcala
words = lines.select(explode(split(col("value"), " ")).alias("word"))

word_counts = words.groupBy("word").count().sort(desc("count"))
#value ile azalansiraya gore topla

query = word_counts.writeStream \
        .outputMode("complete") \
        .format("console") \
        .start()
#aggreagtion icin append
#okunacagi yer console


query.awaitTermination()
#streaming baslasin

KeyboardInterrupt: 

In [ ]:
query.stop()

# READ FROM CONSOLE

In [1]:
import findspark 
findspark.init("/Users/resitkadir/spark/spark-2.4.6/")
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder \
        .master("local[4]") \
        .appName("ReadFromConsole") \
        .config("spark.driver.memory","2g") \
        .config("spark.executor.memory", "4g") \
        .getOrCreate()

lines = spark.readStream \
        .format("socket") \
        .option("host","localhost") \
        .option("port",9999) \
        .load()

words = lines.select(explode(split(col("value"), " ")).alias("value"))

word_counts = words.groupBy("value").count().sort(desc("count"))

# kafka dizinine dosyasının bulunduğu dizine geçilir
# gitBash ile nc -lk 9999 
# komutu çalıştırılır ve elle kelimeler girilmeye başlanır.

query = word_counts.writeStream \
      .outputMode("complete") \
      .format("console") \
      .start()

query.awaitTermination()

# Sonuçlar console üzerinden gözlemlenir.

StreamingQueryException: 'Connection refused (Connection refused)\n=== Streaming Query ===\nIdentifier: [id = 7888c434-8c2d-4698-a0f1-bb20adda0875, runId = 662e65ab-06fa-4c56-b6c4-f74d945ae9b9]\nCurrent Committed Offsets: {}\nCurrent Available Offsets: {TextSocketV2[host: localhost, port: 9999]: -1}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nSort [count#7L DESC NULLS LAST], true\n+- Aggregate [value#3], [value#3, count(1) AS count#7L]\n   +- Project [value#3]\n      +- Generate explode(split(value#0,  )), false, [value#3]\n         +- StreamingExecutionRelation TextSocketV2[host: localhost, port: 9999], [value#0]\n'

In [ ]:
query.stop()

# ReadFromConsoleToDF

In [1]:
import findspark
findspark.init("/Users/resitkadir/spark/spark-3.0.0")
from warnings import filterwarnings
filterwarnings('ignore')
import pyspark.sql.functions as f

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


spark = SparkSession.builder \
        .master("local[4]") \
        .appName("ReadFromConsoleToDF") \
        .config("spark.driver.memory","2g") \
        .config("spark.executor.memory","4g") \
        .getOrCreate()

lines = spark.readStream \
.format("socket") \
.option("host","localhost") \
.option("port", 9999) \
.load()

words = lines.withColumn("column1", f.split(f.col("value"), "\s+")[0]) \
                .withColumn("column2", f.split(f.col("value"), "\s+")[1]) \
                .withColumn("column3", f.split(f.col("value"), "\s+")[2]) \
                .drop("value")

#ncat calistir
# kafka dizinine dosyasının bulunduğu dizine geçilir
# gitBash ile nc -lk 9999 
# komutu çalıştırılır ve elle kelimeler girilmeye başlanır.
query = words.writeStream \
        .outputMode("append") \
        .format("console") \
        .start()

query.awaitTermination()

StreamingQueryException: Connection refused (Connection refused)
=== Streaming Query ===
Identifier: [id = b9963c2d-0a59-400b-84ff-af923084d974, runId = fdfb7eef-0302-4d47-94bd-c3b367c3cb9a]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: localhost, port: 9999]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource ConsoleWriter[numRows=20, truncate=true]
+- Project [column1#2, column2#5, column3#9]
   +- Project [value#0, column1#2, column2#5, split(value#0, \s+, -1)[2] AS column3#9]
      +- Project [value#0, column1#2, split(value#0, \s+, -1)[1] AS column2#5]
         +- Project [value#0, split(value#0, \s+, -1)[0] AS column1#2]
            +- StreamingDataSourceV2Relation [value#0], org.apache.spark.sql.execution.streaming.sources.TextSocketTable$$anon$1@1cf43bd9, TextSocketV2[host: localhost, port: 9999]
